# Building the QUBO is actually the most complicated part!

# Example: Graph Partitioning

In [1]:
# Copyright 2019 D-Wave Systems, Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In this demo, we explore the graph partitioning problem. 

**Example of application** suppose that we have a group of people that we need to split into two **equal-sized** teams. Between some pairs of people there is a friendly relationship. We want to have as few friends on opposite teams as possible. 

We represent our group of people as a graph (a node/vertex for each person), and draw an edge between people that are friends, we have a model of our network. If we look for a **minimum cut in our graph, then we are looking for a way to split the nodes into two groups so that there are as few edges as possible between the groups**. In our group of people, this means that we have two teams with as few friends as possible on different teams.

We will run the demo to find the best way of splitting a 40-node graph (an Erdos-Renyi random graph that probabilistically creates an edge between 20% of its node pairs) into two subsets to minimize the number of cut edges.

# Code overview

The code implements a QUBO formulation of this problem.

The answer that we are looking for is a partition of the nodes in the graph, so we will assign a binary variable for each node, i.e. variable $x_i$ denotes whether node $i$ is in one subset (call it Subset $0$) or the other (Subset $1$).

**The objective function that we want should minimize the number of cut edges.** 

The table below shows how we count the cut edges for a given graph partition (assignment of values to our binary variables). Columns $x_i$ and $x_j$ are two nodes; column edge $(i, j)$ represents an edge between these two nodes. We only want to count an edge if the endpoints are in different subsets, and so we assign a $1$ for the edge column in this case and a $0$ otherwise.

<img src="files/tabella.png" alt="" style="width: 200px;"/>

From this table, we see that we can use the expression x_i+x_j-2x_ix_j to calculate the edge column in our table. Now for our entire graph, our objective function can be written as shown below, where the sum is over all edges in the graph, denoted by E.



**So far it's easy! It is the same as the Max Cut problem, right?**

## Let's consider the constaints!

**SIZE**:Subset $0$ and Subset $1$ must have the same sizes. 

To ensure the two subsets have the same size, we enforce a constraint that Subset $1$ has size equal to half of all nodes in the graph.  $V$ represents the set of all nodes in the graph.

<img src="files/1.png" alt="" style="width: 150px;"/>

For a QUBO, we need our constraints to be represented by mathematical expressions that are satisfied at their minimum value. For this constraint, we can use the following expression that has a minimum value of 0 that occurs when Subset 1 has size exactly $|V|/2$.

<img src="files/2.png" alt="" style="width: 150px;"/>

To simplify this expression and determine the coefficients for our QUBO equation, we first multiply out the expression.

<img src="files/3.png" alt="" style="width: 350px;"/>

Next we can simplify this expression down to linear and quadratic terms for our QUBO. Recall that for binary variables we can replace any squared term with a linear term (since 0^2=0 and 1^2=1), and we can remove any constant terms in our QUBO. This results in the following final expression for our constraint.

<img src="files/4.png" alt="" style="width: 350px;"/>

To combine our objective and constraints into a single QUBO expression, **we simply add together the objective function and our constraint (multiplied by gamma, the Lagrange parameter)**.

<img src="files/5.png" alt="" style="width: 700px;"/>





In [2]:
# ------ Import necessary packages ----
import networkx as nx
from collections import defaultdict
from itertools import combinations
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import math



In [6]:
# ------- Set up our graph -------
G = nx.gnp_random_graph(40, 0.2)

print("Graph on {} nodes created with {} out of {} possible edges.".format(len(G.nodes), len(G.edges), len(G.nodes) * (len(G.nodes)-1) / 2))


Graph on 40 nodes created with 164 out of 780.0 possible edges.


There are three parameters to be set by the user in this code: **chain strength, number of reads, and gamma**. 

Since this is a relatively large problem, we set a large **number of reads (num_reads = 1000)**. 

For **chain strength**, we examine the entries in our Q matrix and choose a relatively large number to enforce chains in our embedding. For this problem, our largest matrix entries are the linear coefficients (approximately size gamma*(1-1*len(G.nodes))) and so a value of gamma*len(G.nodes) is chosen for chain_strength.

For **gamma**, our Lagrange parameter, a good number to start with is an estimate for your optimal objective function value. 

NB. **gamma** is not easy to set... 

Hint: Since we are utilizing Erdos-Renyi random graphs, we assume that our graph will have about 20% of the number of cut edges that a complete graph would have (since our edge probability in G is also 20%). Since a complete graph on 40 nodes has (n(n-1)):2=780 edges, and any partition into equal subsets results in a cut of size 400 (20 nodes in each subset with all 20x20 edges between subsets cut), we can estimate our objective function value. We are examining an Erdos-Renyi random graph with an edge probability of 20%, we can estimate a cut of size approximately 20% of 400, or 80 edges. This leads us to try a Lagrange parameter of 80, which is implemented in our program with gamma=80



In [7]:
# ------- Set tunable parameters -------
# 
num_reads = 1000
gamma = 80 

# Set chain strength
chain_strength = gamma*len(G.nodes)

In [8]:
# ------- Set up our QUBO dictionary -------

# Initialize our Q matrix
Q = defaultdict(int)

# Fill in Q matrix
for u, v in G.edges:
    Q[(u,u)] += 1
    Q[(v,v)] += 1
    Q[(u,v)] += -2

for i in G.nodes:
    Q[(i,i)] += gamma*(1-len(G.nodes))

for i, j in combinations(G.nodes, 2):
	Q[(i,j)] += 2*gamma



In [9]:
# ------- Run our QUBO on the QPU -------

# Run the QUBO on the solver from your config file
sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample_qubo(Q,
                               chain_strength=chain_strength,
                               num_reads=num_reads,
                               label='Example - Graph Partitioning')

# See if the best solution found is feasible, and if so print the number of cut edges.
sample = response.record.sample[0]

# In the case when n is odd, the set may have one more or one fewer nodes
if sum(sample) in [math.floor(len(G.nodes)/2), math.ceil(len(G.nodes)/2)]:
    num_cut_edges = 0
    for u, v in G.edges:
        num_cut_edges += sample[u] + sample[v] - 2*sample[u]*sample[v]
    print("Valid partition found with", num_cut_edges, "cut edges.")
else:
    print("Invalid partition.")

Valid partition found with 71 cut edges.


References https://www.frontiersin.org/articles/10.3389/fphy.2014.00005/full

https://cloud.dwavesys.com/leap/example-details/222052595